# Application My Credit

>> Descriptions détaillées des colonnes données clients bancaires :

1 - âge (numérique)

2 - emploi : type d'emploi (catégorique : "admin.", "inconnu", "chômeur", "gestion", "femme de ménage", "entrepreneur", "étudiant", "col bleu", "indépendant", "retraité", "technicien", "services")

3 - matrimonial : état civil (catégorique : "marié", "divorcé", "célibataire" ; attention : "divorcé" signifie divorcé ou veuf)

4 - éducation (catégorique : "inconnu", "secondaire", "primaire", "tertiaire")

5 - défaut : le crédit est-il en défaut ? (binaire : "oui", "non")

6 - solde : solde annuel moyen, en euros (numérique) 

7 - logement : a-t-il un prêt logement ? (binaire : "oui", "non")

8 - prêt : avez-vous un prêt personnel ? (binaire : "oui", "non")

>> lié au dernier contact de la campagne en cours :

9 - contact : type de communication du contact (catégorique : "inconnu", "téléphone", "cellulaire")

10 - jour : dernier jour de contact du mois (numérique)

11 - mois : dernier mois de contact de l'année (catégorique : "jan" , "feb", "mar", …, "nov", "dec")

12 - durée : durée du dernier contact, en secondes (numérique)

>> autres attributs :

13 - campagne : nombre de contacts effectués pendant cette campagne et pour cette client (numérique, inclut le dernier contact)

14 - pdays : nombre de jours écoulés après que le client a été contacté pour la dernière fois lors d'une campagne précédente (numérique, -1 signifie que le client n'a pas été contacté auparavant)

15 - précédent :nombre de contacts effectués avant cette campagne et pour ce client (numérique)

16 - poutcome : résultat de la campagne marketing précédente (catégorique : "inconnu", "autre", "échec", "succès")

>> Variable de sortie (cible souhaitée) :

17 - y - le client a-t-il obtenu un crédit ? (binaire : "oui", "non")

In [1]:
import pandas as pd

df = pd.read_csv('test.csv', delimiter=';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


In [2]:
from sklearn.preprocessing import LabelEncoder
import json

encod = LabelEncoder()

data_json = {}
cat_col_list = list(df.select_dtypes(include=['object']).columns)

for col in cat_col_list: 
    df[col] = encod.fit_transform(df[col])
    data_json[col] = {encod.classes_[i]:i for i in range(len(encod.classes_))}

with open('data.json', 'w') as fp:json.dump(data_json, fp)

In [3]:
y = df.y
X = df.drop('y', axis=1)

In [4]:
data_json

{'job': {'admin.': 0,
  'blue-collar': 1,
  'entrepreneur': 2,
  'housemaid': 3,
  'management': 4,
  'retired': 5,
  'self-employed': 6,
  'services': 7,
  'student': 8,
  'technician': 9,
  'unemployed': 10,
  'unknown': 11},
 'marital': {'divorced': 0, 'married': 1, 'single': 2},
 'education': {'primary': 0, 'secondary': 1, 'tertiary': 2, 'unknown': 3},
 'default': {'no': 0, 'yes': 1},
 'housing': {'no': 0, 'yes': 1},
 'loan': {'no': 0, 'yes': 1},
 'contact': {'cellular': 0, 'telephone': 1, 'unknown': 2},
 'month': {'apr': 0,
  'aug': 1,
  'dec': 2,
  'feb': 3,
  'jan': 4,
  'jul': 5,
  'jun': 6,
  'mar': 7,
  'may': 8,
  'nov': 9,
  'oct': 10,
  'sep': 11},
 'poutcome': {'failure': 0, 'other': 1, 'success': 2, 'unknown': 3},
 'y': {'no': 0, 'yes': 1}}

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
# XGBoost
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

import pickle

pickle.dump(clf, open('model-xg.pkl', 'wb'))

In [7]:
clf.score(X_test, y_test)

0.8939226519337017